In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
def plot_spectrogram(spec, freqs, max_freq=5000, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
        fig.set_size_inches(10, 5)
        
    ax.pcolorfast(range(spec.shape[1]), freqs[freqs<max_freq], np.log10(spec[freqs<max_freq]))
    ax.set_ylabel('frequency [Hz]')
    return ax

# First study

In [ ]:
from evaluate_data import read_df_from_wav, get_spectrogram

freq = 1750
position_dict = {
    3: 'upper',
    #4: 'upper',
    5: 'current',
    #6: 'current',
    7: 'lower',
    #8: 'lower',
    14: 'lowest',
    #15: 'lowest',
}

In [ ]:
base_filename = f'../experiments/2021_01_07_snr_study/export/motors_nosnr_noprops_mono{freq}'

df_specs = pd.DataFrame(columns=['number', 'spec', 'freqs'])
for number, position_name in position_dict.items():
    appendix = '' if number==0 else f'_{number}'
    try:
        filename = f'{base_filename}{appendix}.wav'
        df_wav = read_df_from_wav(filename)
    except FileNotFoundError:
        print('skipping', filename)
        continue
    spec = get_spectrogram(df_wav)
    freqs = df_wav.iloc[0].frequencies
    
    ax = plot_spectrogram(spec, freqs, max_freq=5000)
    ax.set_title(position_name)
    
    df_specs.loc[len(df_specs), :] = {
        'number': number,
        'spec':spec,
        'freqs':freqs
    }

In [ ]:
time_window_dict =  {
    'buzzer only': range(10, 20),
    'buzzer and props': range(80, 90)
}

fig, axs = plt.subplots(1, len(df_specs), sharey=True)
fig.set_size_inches(20, 5)
for title, time_window in time_window_dict.items():
    
    fig, ax  = plt.subplots()
    fig.set_size_inches(20, 5)
    
    for i, row in df_specs.iterrows():
        spec = row.spec
        freqs = row.freqs

        max_freq = 16000

        spec_window = spec[:, time_window] 
        spec_window = spec_window[freqs<max_freq, :]

        ax.semilogy(freqs[freqs<max_freq], np.mean(spec_window, axis=1), label=position_dict[row.number], 
                    color=f'C{i}')
        axs[i].semilogy(freqs[freqs<max_freq], np.mean(spec_window, axis=1), color=f'C{i}')
        axs[i].set_title(position_dict[row.number])
    ax.legend(loc='upper right')
    ax.set_title(title)

In [ ]:
fig = plt.figure()
fig.set_size_inches(15, 15)
plt.loglog(freqs[freqs<max_freq], np.mean(spec_window, axis=1), color=f'C{i}')
plt.grid(which='both')

# Sweep study

In [ ]:
exp_dict = {
    '45000': 'motors_nosnr_noprops_None.wav',
    '55000': 'motors_nosnr_noprops_None_55000.wav',
    'mono': 'nomotors_nosnr_noprops_mono3125.wav',
    'mono 55000': 'motors_nosnr_noprops_mono3125_55000.wav',
    'mono 45000': 'motors_nosnr_noprops_mono3125.wav',
    'sweep': 'nomotors_nosnr_noprops_sweep.wav',
    'sweep 45000': 'motors_nosnr_noprops_sweep.wav',
}

freq = 3125
#base_filename = f'../experiments/2021_01_14_snr_study/export/'
#base_filename = f'../experiments/2021_01_15_snr_study/export/'
base_filename = f'../experiments/2021_01_15_snr_study_2/export/'

df_specs = pd.DataFrame(columns=['number', 'spec', 'freqs'])
for title, fname in exp_dict.items():
    try:
        filename = base_filename + fname
        df_wav = read_df_from_wav(filename)
    except FileNotFoundError:
        print('skipping', filename)
        continue
    spec = get_spectrogram(df_wav)
    freqs = df_wav.iloc[0].frequencies
    
    ax = plot_spectrogram(spec, freqs, max_freq=5000)
    ax.set_title(title)
    
    df_specs.loc[len(df_specs), :] = {
        'title': title,
        'spec':spec,
        'freqs':freqs
    }